# Experiments

### Setup

In [11]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = "" # API key is sensitive to share
os.environ["LANGSMITH_API_KEY"] = "" # API key is sensitive to share
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [12]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

False

In [13]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install scikit-learn
!pip install openai

Here is the RAG Application that we've been working with throughout this course

In [14]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [15]:
from langsmith import evaluate, Client

client = Client()
dataset_id = "e15cc577-5260-4be3-a7ab-258585878ade"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data="e15cc577-5260-4be3-a7ab-258585878ade",
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-49670729' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=83d2b4bf-68bb-4776-8fab-38b29bc6cf15




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.602799,1868b359-b3ee-4908-8655-ae9f0184bc56,4bee5b38-2f91-4c5d-b2e9-366969fddf69
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.349986,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,0e236b6a-bb65-4c04-9e94-0d59c9793d21
2,Does LangSmith support offline evaluation?,"No, LangSmith primarily supports online evalua...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.066704,5524be6a-278d-4d44-830c-49aa9a66fe4d,6354c3a8-8c22-4094-8591-aa15ccbcc24b
3,How do I pass metadata in with @traceable?,You can pass metadata in with `@traceable` by ...,None,You can pass metadata with the @traceable deco...,1,2.994770,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,076ae590-38d2-4050-94e7-707265435a8b
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.182346,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,8dc1a8ba-2804-4558-a2fb-e6d86c9d3f3b
5,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.082827,b317d976-030d-4bdc-ad85-71bde482f46c,b837037b-49cf-48e1-a504-b4118a797d4c
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",1,2.284648,b4fd0e07-e10c-4462-bc59-1b1b24e76445,03ac605e-fcda-4792-ad63-482d796c281a
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.466669,ca92d9ca-b757-4ec9-b317-33c17f60046f,0bda4b68-35a6-4482-9883-ae40102a99de
8,Can LangSmith be used for finetuning and model...,LangSmith is designed for building production-...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.301928,ce8a6050-8b40-4f24-b1c7-f81b3e4f23c0,50f32606-3fb5-4631-a95c-23166a26ba14
9,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,2.305500,fb838522-26d9-4966-b272-b687b41c4df7,544d85b5-3ec0-4a0a-be25-ed5ef02ca231


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [16]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-e885032c' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=653f3d11-840b-4733-90f5-449fd9d44b71




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in the ...,None,To trace with the @traceable decorator in Pyth...,1,1.884763,1868b359-b3ee-4908-8655-ae9f0184bc56,81cebcc3-ba65-4a10-bac7-5c3f5480af82
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,4.633943,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,4e8cfca3-6ec4-49ea-9322-a42fbbff71c5
2,Does LangSmith support offline evaluation?,The provided context does not mention offline ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.329311,5524be6a-278d-4d44-830c-49aa9a66fe4d,a24da228-9037-4471-8257-61b9fd458103
3,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` annotat...,None,You can pass metadata with the @traceable deco...,0,2.734939,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,a66dace4-287c-4dc1-89a6-86bf6c10b516
4,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.962811,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,bc7589ce-8216-4134-8cbc-782df26eb424
5,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.913820,b317d976-030d-4bdc-ad85-71bde482f46c,b24af1da-9fb9-4d12-b270-f8ac881a262b
6,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.310211,b4fd0e07-e10c-4462-bc59-1b1b24e76445,e344f274-7777-42b6-8fc8-9ac6e31ae83b
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,0.948066,ca92d9ca-b757-4ec9-b317-33c17f60046f,5eb7a93a-c894-470e-a829-5c80c4a60ed6
8,Can LangSmith be used for finetuning and model...,LangSmith is not specifically designed for fin...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.215676,ce8a6050-8b40-4f24-b1c7-f81b3e4f23c0,d99d73a4-7d00-4820-a748-4b9455b035d7
9,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.284617,fb838522-26d9-4966-b272-b687b41c4df7,4f8692b3-c28f-4114-935c-876cc446c84a


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [37]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name),  # latest version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-72472201' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=f338fef8-20d2-4c7c-ab59-72a9b5fc4403




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.825053,1868b359-b3ee-4908-8655-ae9f0184bc56,716292ba-7c57-44e4-a724-11f38d142fff
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.788648,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,db356a09-d8a7-41d4-ba7c-89175cebaf93
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.136167,b4fd0e07-e10c-4462-bc59-1b1b24e76445,da878f13-1d80-4d2c-a9b6-96435eb75bd9
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.795483,ca92d9ca-b757-4ec9-b317-33c17f60046f,dff080d8-83dc-4d70-9add-19ae0ca8c25c
4,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.357341,fb838522-26d9-4966-b272-b687b41c4df7,f4ec6e91-a5f5-4c02-aa7b-72e16d0df9a6
5,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,3.039343,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,6fbc7335-90e5-4b64-8531-2ade272fd8a0
6,Does LangSmith support offline evaluation?,LangSmith does not explicitly mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.055085,5524be6a-278d-4d44-830c-49aa9a66fe4d,d785fc78-6832-4706-9ff2-75963920de9f
7,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,2.054812,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,d44e7b8b-d580-498d-99ef-15ecbc1cb182
8,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.922339,b317d976-030d-4bdc-ad85-71bde482f46c,82e938a8-d51a-4aa2-b5da-2f976026e469
9,Can LangSmith be used for finetuning and model...,LangSmith is designed for LLM observability an...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.406130,ce8a6050-8b40-4f24-b1c7-f81b3e4f23c0,4742df03-0c5f-48e3-ba78-0e4aa209b27e


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [22]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-4d215324' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=e00749b3-8f68-4aa9-aaa3-6ec8744175a3




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,"To trace with the `@traceable` decorator, simp...",None,To trace with the @traceable decorator in Pyth...,1,1.546777,1868b359-b3ee-4908-8655-ae9f0184bc56,816c5aed-7078-4f23-b2cc-74b6cadbc6cc
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.848385,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,9d58ba58-bed7-4a16-a042-2e9ebfaeebc3
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.412161,b4fd0e07-e10c-4462-bc59-1b1b24e76445,2dbb1f36-25e3-4727-9a79-5b840ed65784
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.953147,ca92d9ca-b757-4ec9-b317-33c17f60046f,d853e171-edea-4edb-a91c-da07ae33698a
4,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.352017,fb838522-26d9-4966-b272-b687b41c4df7,2016b73a-e50f-48a7-acae-8b289f352c52


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [31]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name= dataset_name,
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "1868b359-b3ee-4908-8655-ae9f0184bc56",
            "ca92d9ca-b757-4ec9-b317-33c17f60046f"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-f4719103' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=3e3cf3bf-ba35-40e7-923a-8d6eec021476




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.562100,1868b359-b3ee-4908-8655-ae9f0184bc56,81f4bf4e-a18b-424b-af2f-9e2f2a391145
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.833107,ca92d9ca-b757-4ec9-b317-33c17f60046f,b1a3a30c-cb2e-47d3-8129-c65b8aafa73e


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [24]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-23de7023' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=5829d9bb-32ac-4edb-b3ce-67174fdeaa89




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I trace with the @traceable decorator?,To trace with the @traceable decorator using L...,None,To trace with the @traceable decorator in Pyth...,1,1.349770,1868b359-b3ee-4908-8655-ae9f0184bc56,8a009f34-a3b1-4937-81e5-08f8a60225aa
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.409832,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,588c60ea-4b2c-400a-b787-9a79c094d87f
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.286207,b4fd0e07-e10c-4462-bc59-1b1b24e76445,bc3c51a4-1107-45c0-ac77-0124bde1325c
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.565430,ca92d9ca-b757-4ec9-b317-33c17f60046f,e0cc2cbd-a52b-4cb9-9f6c-74b3021f98fa
4,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.252395,fb838522-26d9-4966-b272-b687b41c4df7,5e5edf68-982d-4f8d-8a04-06af9ebd3d26
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing with LangSmith using LangCha...,None,To set up tracing to LangSmith while using Lan...,0,1.802486,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,f11469ff-4169-4110-b84d-449abc2b3e1a
6,Does LangSmith support offline evaluation?,The context provided does not specify any offl...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.758773,5524be6a-278d-4d44-830c-49aa9a66fe4d,c3cda744-e429-40b2-ae74-17074efa1d64
7,How do I pass metadata in with @traceable?,You can pass metadata into traces using the `e...,None,You can pass metadata with the @traceable deco...,1,1.451153,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,347f4c01-f1af-46cc-ade5-b642d569d762
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.516988,b317d976-030d-4bdc-ad85-71bde482f46c,c17e83d6-bed6-4e1d-a024-a3bc277e0330
9,Can LangSmith be used for finetuning and model...,LangSmith is primarily a platform for LLM obse...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.611826,ce8a6050-8b40-4f24-b1c7-f81b3e4f23c0,0a32f762-6c8f-4cfa-8e6b-e9b178f99ba8


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [25]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-3740c747' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=a8b424ac-d745-46d8-ab2d-38e83e11e566




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-dG527CqlOVsdPysVPpv4BJC5 on tokens per min (TPM): Limit 30000, Used 29647, Requested 1566. Please try again in 2.426s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/tmp/ipython-input-862480817.py", line 5, in target_function
    return langsmith_rag(inputs["question"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 111, in langsmith_rag
    response = generate_response(question, documents)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 83, in generate_re

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id,feedback.wrapper
0,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation....",None,"Yes, LangSmith supports online evaluation as a...",1.0,1.297558,b4fd0e07-e10c-4462-bc59-1b1b24e76445,1a18d69b-df81-4208-b23e-1eb05762409d,NaN
1,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1.0,1.386692,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,6d5f4f0c-6c1e-4e8d-9f94-56e765b0fd9b,NaN
2,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1.0,1.566277,1868b359-b3ee-4908-8655-ae9f0184bc56,1802f108-9dff-4757-9518-58724e163af2,NaN
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1.0,1.283892,ca92d9ca-b757-4ec9-b317-33c17f60046f,f345665d-d552-445d-9e05-71fe18e41884,NaN
4,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1.0,1.205374,fb838522-26d9-4966-b272-b687b41c4df7,42fc1fab-b701-4ab4-9079-70a2674c6d62,NaN
5,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,1.0,1.985617,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,581df385-1da3-43a5-b087-eb48b3b9513b,NaN
6,How do I pass metadata in with @traceable?,To pass metadata using the `@traceable` decora...,None,You can pass metadata with the @traceable deco...,1.0,2.100143,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,6a8629af-0c84-4820-8e9e-2c9933456a2e,NaN
7,Does LangSmith support offline evaluation?,The provided context does not mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1.0,2.738034,5524be6a-278d-4d44-830c-49aa9a66fe4d,83438f74-54bc-486b-8263-8ef4469dfc89,NaN
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1.0,1.924330,b317d976-030d-4bdc-ad85-71bde482f46c,31f6158b-3b55-4696-b8c7-1e0f9ccc1985,NaN
9,Is there a Javascript Langsmith SDK,"Yes, there is a JavaScript/TypeScript SDK for ...",None,"Yes, there is a Javascript Langsmith SDK!",1.0,1.171288,cc388aab-a574-48d3-b1ca-050b4e55eca4,54c4c12c-66e4-4e05-9b07-ec645049dca4,NaN


##### Metadata

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [26]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-17757c5a' at:
https://smith.langchain.com/o/c1b67c13-bec2-4f98-8bbf-502c78449186/datasets/e15cc577-5260-4be3-a7ab-258585878ade/compare?selectedSessions=a0db23a0-30e5-4332-9084-2ee74c9f323e




0it [00:00, ?it/s]

ERROR:langsmith.evaluation._runner:Error running target function: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o in organization org-dG527CqlOVsdPysVPpv4BJC5 on tokens per min (TPM): Limit 30000, Used 28376, Requested 1641. Please try again in 34ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langsmith/evaluation/_runner.py", line 1923, in _forward
    fn(*args, langsmith_extra=langsmith_extra)
  File "/tmp/ipython-input-862480817.py", line 5, in target_function
    return langsmith_rag(inputs["question"])
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 111, in langsmith_rag
    response = generate_response(question, documents)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-252468420.py", line 83, in generate_resp

,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id,feedback.wrapper
0,How can I trace with the @traceable decorator?,To trace with the `@traceable` decorator in Py...,None,To trace with the @traceable decorator in Pyth...,1.0,1.670214,1868b359-b3ee-4908-8655-ae9f0184bc56,a4b6af3e-c908-4a8b-9514-f697d12c3b7b,NaN
1,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1.0,2.967447,78e2e0eb-f8fb-4d8e-b6a5-e6e1ca31808b,b3f191bf-e2a5-4259-a76e-4646d54ab845,NaN
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1.0,1.362835,b4fd0e07-e10c-4462-bc59-1b1b24e76445,126f852e-105e-4d45-a484-0fe4b726bbc8,NaN
3,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1.0,1.039020,ca92d9ca-b757-4ec9-b317-33c17f60046f,bd328e49-c2fe-4d5e-b19c-d7dc960c5dbd,NaN
4,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1.0,1.097408,fb838522-26d9-4966-b272-b687b41c4df7,b83cbc77-897f-4caa-92eb-0c3e9ac954e9,NaN
5,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0.0,5.182827,3d54ef8e-97e6-482d-97a4-2fb5171ecb1d,6be7257e-dbdc-4f69-946c-fe25e86ff565,NaN
6,Does LangSmith support offline evaluation?,The context provided does not mention offline ...,None,"Yes, LangSmith supports offline evaluation thr...",1.0,1.762790,5524be6a-278d-4d44-830c-49aa9a66fe4d,3d73e6a4-51ef-4e79-ab6d-46a05aa0f1c2,NaN
7,How do I pass metadata in with @traceable?,You can pass metadata in with the `@traceable`...,None,You can pass metadata with the @traceable deco...,1.0,3.568063,5a2f44cc-10d8-4d38-a734-3ddfea60fe24,e053d33d-37d5-4cd0-a442-0a5bf1893eef,NaN
8,How do I create user feedback with the LangSmi...,None,"RateLimitError(""Error code: 429 - {'error': {'...",To create user feedback with the LangSmith SDK...,NaN,1.040469,b317d976-030d-4bdc-ad85-71bde482f46c,087eb7cf-1766-4e51-a2dd-2653a4758edc,NaN
9,Can LangSmith be used for finetuning and model...,"No, LangSmith is primarily focused on LLM obse...",None,"Yes, LangSmith can be used for fine-tuning and...",1.0,1.844840,ce8a6050-8b40-4f24-b1c7-f81b3e4f23c0,afc3c366-430f-4451-bf42-f5a6834dede9,NaN
